In [ ]:
from google.colab import drive
drive.mount('/content/drive')

BASE = "/content/drive/MyDrive/Table Tennis Analysis"
print("Mounted")
!pip -q install opencv-python numpy pandas scipy moviepy
import os

print(os.listdir(BASE))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mounted
['Literature Survey', 'Datasets']


In [ ]:
import json
import pandas as pd

# 🔹 Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 🔹 Path to your JSON file
json_path = "/content/drive/MyDrive/Table Tennis Analysis/Datasets/Segment Analysis/ball_markup_game1.json"

# 🔹 Load JSON
with open(json_path, "r") as f:
    data = json.load(f)

# Convert dict of dicts → list of dicts
records = []
for frame, coords in data.items():
    x, y = coords["x"], coords["y"]
    records.append({"frame": int(frame), "x": x, "y": y})

# Convert to DataFrame
df = pd.DataFrame(records).sort_values("frame")

# 🔹 Define bounding box size (radius around ball)
radius = 10

# Skip invalid coordinates (-1, -1)
df = df[(df["x"] >= 0) & (df["y"] >= 0)]

# Compute bounding boxes
df["x_min"] = df["x"] - radius
df["y_min"] = df["y"] - radius
df["x_max"] = df["x"] + radius
df["y_max"] = df["y"] + radius

# 🔹 Save results
output_path = "/content/drive/MyDrive/ball_bounding_boxes.csv"
df.to_csv(output_path, index=False)

print("Bounding boxes saved to:", output_path)
print(df.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Bounding boxes saved to: /content/drive/MyDrive/ball_bounding_boxes.csv
   frame    x    y  x_min  y_min  x_max  y_max
0     14  506  523    496    513    516    533
1     15  523  529    513    519    533    539
2     16  537  534    527    524    547    544
3     17  552  541    542    531    562    551
4     18  566  544    556    534    576    554


In [ ]:
import json
import cv2
import matplotlib.pyplot as plt

# Load bounding box data from JSON
with open("/content/drive/MyDrive/Table Tennis Analysis/Datasets/Segment Analysis/ball_markup_game1.json", "r") as f:
    bbox_data = json.load(f)

def draw_bounding_box(image_path, frame_number):
    # Load the image
    img = cv2.imread(image_path)

    if img is None:
        raise FileNotFoundError(f"Could not load image: {image_path}")

    # Get bounding box for given frame
    if str(frame_number) not in bbox_data:
        raise ValueError(f"No bounding box found for frame {frame_number}")

    bbox = bbox_data[str(frame_number)]  # [x_min, y_min, x_max, y_max]

    x_min, y_min, x_max, y_max = bbox

    # Draw bounding box
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

    # Convert BGR (OpenCV) to RGB (matplotlib)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Show the image
    plt.imshow(img_rgb)
    plt.axis("off")
    plt.show()

# Example usage:
# Make sure "frame_42.jpg" exists in your folder
draw_bounding_box("frame_42.jpg", frame_number=42)


FileNotFoundError: Could not load image: frame_42.jpg

In [ ]:
import cv2
import os
from google.colab import drive
drive.mount('/content/drive')

video_path = "/content/drive/MyDrive/Table Tennis/Videos/game_1.mp4"
output_dir = "/content/drive/MyDrive/Table Tennis/Videos/frames"
os.makedirs(output_dir, exist_ok=True)

cap = cv2.VideoCapture(video_path)
frame_id = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_filename = os.path.join(output_dir, f"frame_{frame_id:05d}.jpg")
    cv2.imwrite(frame_filename, frame)
    frame_id += 1

cap.release()
print(f"Extracted {frame_id} frames to {output_dir}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Extracted 88599 frames to /content/drive/MyDrive/Table Tennis/Videos/frames


In [ ]:
import os
import pandas as pd
from PIL import Image
from google.colab import drive
drive.mount('/content/drive')
# Paths
frames_dir = "/content/drive/MyDrive/Table Tennis/Videos/frames"   # folder with frame_0001.jpg ...
labels_dir = "/content/drive/MyDrive/Table Tennis/labels1"   # output YOLO labels
os.makedirs(labels_dir, exist_ok=True)

# Load ball annotations
# CSV format: frame,x,y,x_min,y_min,x_max,y_max
df = pd.read_csv("/content/drive/MyDrive/Table Tennis/ball_bounding_boxes_game1.csv")

# Group annotations by frame number
annotations = {}
for _, row in df.iterrows():
    fid = int(row["frame"])
    if fid not in annotations:
        annotations[fid] = []
    annotations[fid].append(row)

for frame_file in sorted(os.listdir(frames_dir)):
    if not frame_file.endswith(".jpg"):
        continue

    # Extract frame number from file name (frame_0001.jpg → 1)
    frame_id = int(frame_file.split("_")[-1].split(".")[0])

    frame_path = os.path.join(frames_dir, frame_file)
    img = Image.open(frame_path)
    w, h = img.size

    label_path = os.path.join(labels_dir, frame_file.replace(".jpg", ".txt"))

    if frame_id in annotations:
        with open(label_path, "w") as f:
            for row in annotations[frame_id]:
                x_min, y_min, x_max, y_max = row["x_min"], row["y_min"], row["x_max"], row["y_max"]

                # Convert to YOLO format
                x_center = ((x_min + x_max) / 2) / w
                y_center = ((y_min + y_max) / 2) / h
                box_w = (x_max - x_min) / w
                box_h = (y_max - y_min) / h

                f.write(f"0 {x_center:.6f} {y_center:.6f} {box_w:.6f} {box_h:.6f}\n")
    else:
        # Frame has no annotation → empty label file
        open(label_path, "w").close()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


OSError: [Errno 5] Input/output error: '/content/drive/MyDrive/Table Tennis/Videos/frames'

In [ ]:
import os
import glob
import pandas as pd
from PIL import Image
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Paths
frames_dir = "/content/drive/MyDrive/Table Tennis/Videos/frames"   # folder with frame_0001.jpg ...
labels_dir = "/content/drive/MyDrive/Table Tennis/labels1"         # output YOLO labels
os.makedirs(labels_dir, exist_ok=True)

# Load ball annotations
# CSV format: frame,x,y,x_min,y_min,x_max,y_max
df = pd.read_csv("/content/drive/MyDrive/Table Tennis/ball_bounding_boxes_game1.csv")

# Group annotations by frame number
annotations = {}
for _, row in df.iterrows():
    fid = int(row["frame"])
    if fid not in annotations:
        annotations[fid] = []
    annotations[fid].append(row)

# Get all frame files safely using glob
frame_files = sorted(glob.glob(os.path.join(frames_dir, "*.jpg")))
print("Total frames found:", len(frame_files))

# Process in batches to avoid I/O overload (optional)
batch_size = 1000   # adjust depending on memory/runtime
for i in range(0, len(frame_files), batch_size):
    batch = frame_files[i:i+batch_size]
    print(f"Processing batch {i//batch_size + 1} with {len(batch)} frames...")

    for frame_path in batch:
        frame_file = os.path.basename(frame_path)

        # Extract frame number from file name (frame_0001.jpg → 1)
        frame_id = int(frame_file.split("_")[-1].split(".")[0])
        if frame_id < 72212:
            continue
        # Open image to get size
        img = Image.open(frame_path)
        w, h = img.size

        # Path for YOLO label
        label_path = os.path.join(labels_dir, frame_file.replace(".jpg", ".txt"))

        if frame_id in annotations:
            with open(label_path, "w") as f:
                for row in annotations[frame_id]:
                    x_min, y_min, x_max, y_max = row["x_min"], row["y_min"], row["x_max"], row["y_max"]

                    # Convert to YOLO format (normalized center x,y and width,height)
                    x_center = ((x_min + x_max) / 2) / w
                    y_center = ((y_min + y_max) / 2) / h
                    box_w = (x_max - x_min) / w
                    box_h = (y_max - y_min) / h

                    f.write(f"0 {x_center:.6f} {y_center:.6f} {box_w:.6f} {box_h:.6f}\n")
        else:
            # Frame has no annotation → create empty label file
            open(label_path, "w").close()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total frames found: 88599
Processing batch 1 with 1000 frames...
Processing batch 2 with 1000 frames...
Processing batch 3 with 1000 frames...
Processing batch 4 with 1000 frames...
Processing batch 5 with 1000 frames...
Processing batch 6 with 1000 frames...
Processing batch 7 with 1000 frames...
Processing batch 8 with 1000 frames...
Processing batch 9 with 1000 frames...
Processing batch 10 with 1000 frames...
Processing batch 11 with 1000 frames...
Processing batch 12 with 1000 frames...
Processing batch 13 with 1000 frames...
Processing batch 14 with 1000 frames...
Processing batch 15 with 1000 frames...
Processing batch 16 with 1000 frames...
Processing batch 17 with 1000 frames...
Processing batch 18 with 1000 frames...
Processing batch 19 with 1000 frames...
Processing batch 20 with 1000 frames...
Processing batch 21 with 1000 frames...
Processing bat

In [ ]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')
# Paths
images_path = "/content/drive/MyDrive/Table Tennis/Videos/frames"
labels_path = "/content/drive/MyDrive/Table Tennis/labels1"
output_path = "/content/drive/MyDrive/Table Tennis/dataset"

# Train/Val split ratio
val_ratio = 0.2

# Get all images (assuming .jpg or .png)
image_files = [f for f in os.listdir(images_path) if f.endswith(('.jpg', '.png'))]

# Split train and val
train_files, val_files = train_test_split(image_files, test_size=val_ratio, random_state=42)

# Function to copy files
def copy_files(file_list, split):
    for f in file_list:
        # image
        src_img = os.path.join(images_path, f)
        dst_img = os.path.join(output_path, "images", split, f)
        os.makedirs(os.path.dirname(dst_img), exist_ok=True)
        shutil.copy(src_img, dst_img)

        # label (same name, .txt)
        label_file = os.path.splitext(f)[0] + ".txt"
        src_lbl = os.path.join(labels_path, label_file)
        if os.path.exists(src_lbl):
            dst_lbl = os.path.join(output_path, "labels", split, label_file)
            os.makedirs(os.path.dirname(dst_lbl), exist_ok=True)
            shutil.copy(src_lbl, dst_lbl)

# Copy train and val
copy_files(train_files, "train")
copy_files(val_files, "val")

print(f"✅ Dataset prepared in {output_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

dataset_path = "/content/drive/MyDrive/Table Tennis/dataset/"

# Count images
image_count = 0
for root, dirs, files in os.walk(os.path.join(dataset_path, "images")):
    image_count += len([f for f in files if f.endswith(('.jpg', '.png'))])

# Count labels
label_count = 0
for root, dirs, files in os.walk(os.path.join(dataset_path, "labels")):
    label_count += len([f for f in files if f.endswith('.txt')])

print(f"🖼️ Total images: {image_count}")
print(f"📝 Total labels: {label_count}")


🖼️ Total images: 88599
📝 Total labels: 88599


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Paths
images_path = "/content/drive/MyDrive/Table Tennis/Videos/frames"
labels_path = "/content/drive/MyDrive/Table Tennis/labels1"
output_path = "/content/drive/MyDrive/Table Tennis/dataset"

# Train/Val split ratio
val_ratio = 0.2

# Get all images (assuming .jpg or .png)
image_files = [f for f in os.listdir(images_path) if f.endswith(('.jpg', '.png'))]

# Split train and val (use same random_state to keep consistency)
from sklearn.model_selection import train_test_split
train_files, val_files = train_test_split(image_files, test_size=val_ratio, random_state=42)

# Function to copy files if not already copied
def copy_files(file_list, split):
    for f in file_list:
        # image
        src_img = os.path.join(images_path, f)
        dst_img = os.path.join(output_path, "images", split, f)
        os.makedirs(os.path.dirname(dst_img), exist_ok=True)
        if not os.path.exists(dst_img):  # ✅ Skip if already copied
            shutil.copy(src_img, dst_img)

        # label
        label_file = os.path.splitext(f)[0] + ".txt"
        src_lbl = os.path.join(labels_path, label_file)
        dst_lbl = os.path.join(output_path, "labels", split, label_file)
        os.makedirs(os.path.dirname(dst_lbl), exist_ok=True)
        if os.path.exists(src_lbl) and not os.path.exists(dst_lbl):  # ✅ Skip if already copied
            shutil.copy(src_lbl, dst_lbl)

# Resume copying
copy_files(train_files, "train")
copy_files(val_files, "val")

print(f"✅ Dataset prepared (resumed) in {output_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Dataset prepared (resumed) in /content/drive/MyDrive/Table Tennis/dataset


In [ ]:
# Colab / Python Notebook

# 1. Install Ultralytics YOLOv11
!pip install ultralytics

from ultralytics import YOLO

# 2. Prepare dataset configuration

# Suppose your dataset folder structure is:
# dataset/
#   images/
#     train/
#     val/
#   labels/
#     train/
#     val/

# Make sure your data.yaml file looks something like this:

data_yaml = """
train: /content/drive/MyDrive/Table Tennis/dataset/images/train
val:   /content/drive/MyDrive/Table Tennis/dataset/images/val

# number of classes
nc:  <NUM_CLASSES>

# class names in order, index 0,1,2...
names:
  0: classA
  1: classB
  # add more if needed
"""

# Save data.yaml
with open('/content/data.yaml', 'w') as f:
    f.write(data_yaml)

# 3. Choose model

# Use a pretrained YOLOv11 model (recommended)
model = YOLO('yolo11n.pt')  # “n” = nano; you can also use s, m, l, x depending on your compute

# Or build from scratch:
# model = YOLO('yolo11n.yaml')

# 4. Train

results = model.train(
    data='/content/data.yaml',
    epochs=50,         # adjust
    imgsz=640,         # image size
    batch=16,          # batch size, adjust based on GPU memory
    device=0,          # GPU device id (or 'cpu')
    project='/content/drive/MyDrive/Table Tennis/YOLOv11_results',
    name='table_tennis_detect',
    resume=False,      # True if resuming training
    augment=True       # use augmentations
)

# 5. Validation / test / inference

# After training, you can validate:
val_results = model.val(data='/content/data.yaml')

# Run inference on some images
predictions = model.predict(
    source='/content/drive/MyDrive/Table Tennis/dataset/images/val',
    conf=0.25,   # confidence threshold
    iou=0.45,    # IoU threshold
    save=True
)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.0 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.202 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015

RuntimeError: Dataset '/content/data.yaml' error ❌ /content/data.yaml 'names' length 2 and 'nc: <NUM_CLASSES>' must match.

In [5]:
# ========================
# 1. Install Ultralytics YOLOv11
# ========================
!pip install ultralytics

from ultralytics import YOLO
import os

# ========================
# 2. Mount Google Drive
# ========================
from google.colab import drive
drive.mount('/content/drive')

# ========================
# 3. Paths
# ========================
dataset_path = "/content/drive/MyDrive/Table Tennis/dataset"

# Write a proper data.yaml file
data_yaml = f"""
train: {dataset_path}/images/train
val: {dataset_path}/images/val

nc: 1
names:
  0: ball
"""

with open('/content/data.yaml', 'w') as f:
    f.write(data_yaml)

print("✅ data.yaml created at /content/data.yaml")

# ========================
# 4. Train YOLOv11
# ========================
# You can use yolo11n (nano), yolo11s (small), yolo11m, etc.
# "n" is fastest but less accurate; "s/m" are better if GPU allows
model = YOLO('yolo11n.pt')

results = model.train(
    data='/content/data.yaml',
    epochs=20,          # increase if needed
    imgsz=640,
    batch=16,           # lower if you hit OOM
    device="cpu",           # use GPU
    project='/content/drive/MyDrive/Table Tennis/YOLOv11_results',
    name='ball_detect',
)

# ========================
# 5. Validate Model
# ========================
metrics = model.val(data='/content/data.yaml')

# ========================
# 6. Run Inference on Val Set
# ========================
predictions = model.predict(
    source=f"{dataset_path}/images/val",
    conf=0.25,
    save=True
)

print("✅ Training and Inference Complete!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ data.yaml created at /content/data.yaml
Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (AMD EPYC 7B12)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scal

KeyboardInterrupt: 

In [5]:
!pip install ultralytics

from ultralytics import YOLO
import os

# ========================
# 2. Mount Google Drive
# ========================
from google.colab import drive
drive.mount('/content/drive')

# ========================
# 3. Paths
# ========================
dataset_path = "/content/drive/MyDrive/Table Tennis/dataset"
project_path = "/content/drive/MyDrive/Table Tennis/YOLOv11_results"

# Write a proper data.yaml file
data_yaml = f"""
train: {dataset_path}/images/train
val: {dataset_path}/images/val

nc: 1
names:
  0: ball
"""

with open('/content/data.yaml', 'w') as f:
    f.write(data_yaml)

print("✅ data.yaml created at /content/data.yaml")

# ========================
# 4. Train YOLOv11 with checkpoints
# ========================
model = YOLO('yolo11n.pt')  # or 'yolo11s.pt', 'yolo11m.pt'

results = model.train(
    data='/content/data.yaml',
    epochs=1,
    imgsz=640,
    batch=16,
    device=0,          # change to 0 if Colab GPU is enabled
    project=project_path,
    name='ball_detect',
    resume=True,
    save=True,             # save checkpoints every epoch
    save_period=1          # force saving every 1 epoch
)

# ========================
# 5. Validate Model
# ========================
metrics = model.val(data='/content/data.yaml')

# ========================
# 6. Run Inference on Val Set
# ========================
predictions = model.predict(
    source=f"{dataset_path}/images/val",
    conf=0.25,
    save=True
)

print("✅ Training and Inference Complete!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ data.yaml created at /content/data.yaml
Ultralytics 8.3.203 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, crop_fraction=1.0, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=600, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, label_smoothing=0.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo

AssertionError: yolo11n.pt training to 600 epochs is finished, nothing to resume.
Start a new training without resuming, i.e. 'yolo train model=yolo11n.pt'